# Testes de treinamento incremental

A ideia deste notebook é verificar se treinar os algoritmos de MAB durante as interações de teste geram alguma melhora para eles, comparando-os com os resultados de um algoritmo estático (ALS).

In [1]:
import pandas as pd
from mab2rec import BanditRecommender, LearningPolicy
from mab2rec.pipeline import train
from sklearn.preprocessing import LabelEncoder
import implicit
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [2]:
def initial_train(df, num_users, num_items):
    FACTORS = 10
    K1 = 100
    B = 0.8

    # Cria a matriz esparsa
    sparse_matrix = csr_matrix((df['response'], (df['user_id'], df['item_id'])), shape=(num_users, num_items))
    updated_sparse_matrix = bm25_weight(sparse_matrix, K1=K1, B=B)

    print('Treinando o modelo ALS')
    ALS_model = implicit.als.AlternatingLeastSquares(factors=FACTORS)
    ALS_model.fit(updated_sparse_matrix)

    user_features_list = []

    for user_id in df['user_id'].unique():
        user_factors = ALS_model.user_factors[user_id]
        user_features_list.append([user_id] + list(user_factors))

    df_user_features = pd.DataFrame(user_features_list, columns=['user_id'] + [f'u{i}' for i in range(FACTORS)])

    print('Treinando o modelo LinUCB')
    linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(), top_k=10)
    train(linUCB_model, data=df, user_features=df_user_features)

    print('Treinando o modelo LinGreedy')
    linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(), top_k=10)
    train(linGreedy_model, data=df, user_features=df_user_features)

    return ALS_model, linUCB_model, linGreedy_model, df_user_features, sparse_matrix

In [3]:
def test_ALS(ALS_model, sparse_matrix, df_test):
    print('Testing ALS')
    hits = 0
    for i, interaction in df_test.iterrows():
        ids_recs, _ = ALS_model.recommend(userid=interaction['user_id'], user_items=sparse_matrix[interaction['user_id']], N=10)
        if interaction['item_id'] in ids_recs:
            hits += 1

    print('\nALS RESULTS:')
    print(f'Hits: {hits}')
    print(f'Total: {len(df_test)}')
    print(f'HR: {hits/len(df_test)}\n')

In [4]:
def test_incremental(mab_algo, algo_name, user_features, df_test, df_test_for_evaluation, batch_size):
    print(f'Testing {algo_name}')
    hits = 0

    for i in range(0, len(df_test), batch_size):
        # Fazendo recomendações para teste
        df_batch_test = df_test_for_evaluation.loc[i:i+batch_size-1]
        contexts = df_batch_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        if len(contexts) > 0:
            recomendations = mab_algo.recommend(contexts)
            if isinstance(recomendations, list) and isinstance(recomendations[0], int):
                recomendations = [recomendations]

            df_batch_test = df_batch_test.reset_index(drop=True)

            for j, interaction in df_batch_test.iterrows():
                if interaction['item_id'] in recomendations[j]:
                    hits += 1
        
        # Treinando com o batch
        df_batch_train = df_test.loc[i:i+batch_size-1]
        contexts = df_batch_train.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        mab_algo.partial_fit(df_batch_train['item_id'], df_batch_train['response'], contexts)

    print(f'\n{algo_name} RESULTS:')
    print(f'Hits: {hits}')
    print(f'Total: {len(df_test_for_evaluation)}')
    print(f'HR: {hits/len(df_test_for_evaluation)}\n')

In [5]:
def test(train_size, batch_size):
    df_train = pd.read_csv(train_data)
    df_test = pd.read_csv(test_data)

    df_full = pd.concat([df_train, df_test])

    df_full['user_id'] = LabelEncoder().fit_transform(df_full['user_id'])
    df_full['item_id'] = LabelEncoder().fit_transform(df_full['item_id'])

    num_users = df_full['user_id'].nunique()
    num_items = df_full['item_id'].nunique()

    split_index = int(len(df_full) * train_size)
    df_train = df_full[:split_index]
    df_test = df_full[split_index:]

    df_test = df_test[(df_test['user_id'].isin(df_train['user_id'])) & (df_test['item_id'].isin(df_train['item_id']))]
    df_test = df_test.reset_index(drop=True)
    df_test_for_evaluation = df_test[df_test['response'] == 1]

    ALS_model, linUCB_model, linGreedy_model, df_user_features, sparse_matrix = initial_train(df_train, num_users, num_items)

    test_ALS(ALS_model, sparse_matrix, df_test_for_evaluation)
    test_incremental(linUCB_model, 'LinUCB', df_user_features, df_test, df_test_for_evaluation, batch_size)
    test_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test, df_test_for_evaluation, batch_size)

In [6]:
test(0.9, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.001001119613647461 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 79
Total: 623
HR: 0.12680577849117175

Testing LinUCB

LinUCB RESULTS:
Hits: 43
Total: 623
HR: 0.06902086677367576

Testing LinGreedy

LinGreedy RESULTS:
Hits: 49
Total: 623
HR: 0.07865168539325842



In [7]:
test(0.5, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 929
Total: 4726
HR: 0.1965721540414727

Testing LinUCB

LinUCB RESULTS:
Hits: 421
Total: 4726
HR: 0.08908167583580194

Testing LinGreedy

LinGreedy RESULTS:
Hits: 482
Total: 4726
HR: 0.10198899703766398



In [8]:
test(0.5, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 910
Total: 4726
HR: 0.192551840880237

Testing LinUCB

LinUCB RESULTS:
Hits: 387
Total: 4726
HR: 0.0818874312314854

Testing LinGreedy

LinGreedy RESULTS:
Hits: 435
Total: 4726
HR: 0.09204401184934405



In [9]:
test(0.1, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 833
Total: 8509
HR: 0.09789634504642143

Testing LinUCB

LinUCB RESULTS:
Hits: 547
Total: 8509
HR: 0.06428487483840639

Testing LinGreedy

LinGreedy RESULTS:
Hits: 670
Total: 8509
HR: 0.07874015748031496



In [10]:
test(0.1, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 815
Total: 8509
HR: 0.09578093783053238

Testing LinUCB

LinUCB RESULTS:
Hits: 515
Total: 8509
HR: 0.060524150899048064

Testing LinGreedy

LinGreedy RESULTS:
Hits: 616
Total: 8509
HR: 0.07239393583264779



In [11]:
test(0.05, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0015070438385009766 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 741
Total: 8242
HR: 0.08990536277602523

Testing LinUCB

LinUCB RESULTS:
Hits: 506
Total: 8242
HR: 0.061392865809269594

Testing LinGreedy

LinGreedy RESULTS:
Hits: 575
Total: 8242
HR: 0.06976462023780636



In [12]:
test(0.05, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 743
Total: 8242
HR: 0.09014802232467847

Testing LinUCB

LinUCB RESULTS:
Hits: 527
Total: 8242
HR: 0.06394079107012861

Testing LinGreedy

LinGreedy RESULTS:
Hits: 677
Total: 8242
HR: 0.08214025721912158

